In [1]:
from numba import cuda
from numba import jit
import numpy as np
import time

In [2]:
def maxElement(arr, n):
    max = arr[0]
    for i in range(1, n):
        if arr[i] > max:
            max = arr[i]
    return max

In [3]:
@jit 
def maxElementJit(arr, n):
    max = arr[0]
    for i in range(1, n):
        if arr[i] > max:
            max = arr[i]
    return max

In [4]:
@cuda.jit
def cuda_max_idx(arr, max1):
    tx = cuda.threadIdx.x
    ty = cuda.blockIdx.x
    bw = cuda.blockDim.x
    pos = tx + ty * bw
    
    while(pos < arr.size):
        if arr[pos] > max1[0]:
            max1[0] = arr[pos]
        pos += cuda.blockDim.x * cuda.gridDim.x; 

In [5]:
@cuda.jit
def maxElement_1(arr,n,max_2):
    idx = cuda.grid(1)
    max_1 = arr[0]
    for i in range(1, n):
        if arr[i] > max_1:
            max_1 = arr[i]
    max_2[idx] = max_1

In [6]:

arr = np.random.randint(1000, size = 1000000)

start = time.time()
maxElement(arr, 1000000)
end = time.time() 
s = end - start
print("Sequential Time: ", s)

start = time.time()
maxElementJit(arr, 1000000)
end = time.time() 
j = end - start
print("Jit Time: ", j)

max1 = np.zeros(1)
start = time.time()
cuda_max_idx[128, 64](arr, max1)
end = time.time() 
c = end - start
print("CUDA Time: ", c)

print("speedup-jit: ", s/j)
print("speedup-cuda: ", s/c)

Sequential Time:  0.1724395751953125
Jit Time:  0.5342023372650146
CUDA Time:  0.8939037322998047
speedup-jit:  0.3227982417264607
speedup-cuda:  0.19290620339134942


/usr/local/lib/python3.7/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
